<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/master/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import queue
baseURL = 'https://www.soccerbase.com'
lock = threading.Lock()

In [2]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    lock.acquire()
    print(func.__name__, 'took', end = ' ')
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
    lock.release()
  return timeIt

In [3]:
@timer
def getEveryCompetitionData():
  request = requests.get("https://www.soccerbase.com/players/home.sd")
  soup = BeautifulSoup(request.content, "lxml")
  testing = soup.find('div', id = 'players-menu')
  for div in testing.find_all('div', class_ = 'soccerColumn'):
    if (re.search('Competition', div.text) != None):
      comp = div
  threads = []
  compQ = queue.Queue()
  for competition in comp.find_all('a'):
    t = threading.Thread(target = readCompetitionPage, args = (competition.text, baseURL + competition['href'], compQ))
    threads.append(t)
    t.start()
  for t in threads:
    t.join()
  counter = 1
  while not compQ.empty():
    #print(counter, compQ.get()[0])
    counter += 1

#From competition page of teams
## seasons: dataframe consisted of each season value with the season date

#### getTeamData(team: teamname, url: url has to go to, queue: consisted of map)
goal: 

In [4]:
class Team:
  def __init__(self, team_name, players_df):
    self.team_name = team_name
    self.players_df = players_df
  def get_name(self):
    return self.team_name
  def get_players(self):
    return self.players_df

In [5]:
import pandas as pd
#From player page for a team
def getTeamData(team, url, q):
  #names = [team]
  players_df = pd.DataFrame()
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  try:
    test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
    for elem in test.find_all('b'):
      name = elem.find('a')
      players_df = players_df.append({"link": name["href"], "name" : name.text}, ignore_index = True)
  except AttributeError:
      print(team, 'failed to open')
  team = Team(team, players_df)
  q.put(team)

#### season_selector
teams: map with key is a team name and value is dataframe of player information

In [6]:
def season_selector(seasons):

  value = seasons.iloc[1]["value"]

  url = "https://www.soccerbase.com/tournaments/tournament.sd?tourn_id=" + value
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  try:
    teamNames = soup.find("table", class_ = "table")
    team_df = teams_dataframes(teamNames)
    teams = []
    threads = []
    q = queue.Queue()
    for index, row in team_df.iterrows():
      t = threading.Thread(target = getTeamData, args = (index, "https://www.soccerbase.com" + row["link"], q))
      threads.append(t)
      t.start()
    for t in threads:
      t.join()
    while not q.empty():
      teams.append(q.get()) #teamData[0] == name , teamData[1:] name
    return teams
  except AttributeError:
    pass

In [12]:
import threading
import queue
#@timer
def readCompetitionPage(competition, compURL, compQ):
  request = requests.get(compURL)
  soup = BeautifulSoup(request.content, "lxml")
  seasons = season_dataframes(soup)
  teams = season_selector(seasons)
  if teams != None:
    for team in teams:
      print(team.get_players())

Dataframes
season_dataframes: return the dataframes consisted of seasons and each season values
teams_dataframes: return the dataframe consisted of team names and the link to read data of a team

In [8]:
import re
import pandas as pd
def season_dataframes(soup):
  df = pd.DataFrame()
  list = soup.find("select", id = "seasonSelect")
  for data in list.find_all("option")[1:]:
    df = df.append({"value": data["value"], "date" : data.text}, ignore_index = True)
  df.set_index("date", inplace = True)
  
  return df


In [9]:
def teams_dataframes(soup):
  df = pd.DataFrame()
  list = soup.find("tbody")
  for data in list.find_all("td", class_ = "bull"):
    team = data.find("a")
    df = df.append({"link": team["href"], "name" : team.text}, ignore_index = True)
  df.set_index("name", inplace = True)
  return df

In [13]:
#readCompetitionPage('https://www.soccerbase.com/tournaments/tournament.sd?comp_id=20')
getEveryCompetitionData()

                                   link                  name
0    /players/player.sd?player_id=97497         Michael McGee
1    /players/player.sd?player_id=51847        David Mitchell
2    /players/player.sd?player_id=97965          Liam Allison
3    /players/player.sd?player_id=39848            Jamie Bain
4    /players/player.sd?player_id=53051          Keiran Duffy
5    /players/player.sd?player_id=78286           Craig Howie
6    /players/player.sd?player_id=49940         Martin McNiff
7    /players/player.sd?player_id=71929          Scott Rumsby
8    /players/player.sd?player_id=71309         Barry Cuddihy
9    /players/player.sd?player_id=95745         Raymond Grant
10   /players/player.sd?player_id=63606        Chris Johnston
11   /players/player.sd?player_id=93375           Mark Lamont
12  /players/player.sd?player_id=128541         Daniel Lennon
13   /players/player.sd?player_id=81001             Ross Lyon
14  /players/player.sd?player_id=129614        Kyle McCormack
15  /pla

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-cb2895f2018f>", line 2, in <module>
    getEveryCompetitionData()
  File "<ipython-input-2-c7feaa8ba089>", line 6, in timeIt
    func(*args, **kwargs)
  File "<ipython-input-3-e815a1514f4b>", line 16, in getEveryCompetitionData
    t.join()
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribut

KeyboardInterrupt: ignored